In [2]:
from huggingface_hub import login
login(token="<enter the token here>")

In [15]:
import os
import subprocess
import time
from PIL import Image
from vllm import LLM, SamplingParams

In [3]:
# load the model

llm = LLM(model="google/paligemma-3b-mix-448",  
        trust_remote_code=True,
        gpu_memory_utilization=0.8, 
        max_seq_len_to_capture = 8192)

INFO 07-01 05:06:41 [config.py:2832] Downcasting torch.float32 to torch.float16.
INFO 07-01 05:06:57 [config.py:689] This model supports multiple tasks: {'reward', 'generate', 'score', 'classify', 'embed'}. Defaulting to 'generate'.
INFO 07-01 05:07:02 [config.py:1901] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 07-01 05:07:06 [core.py:61] Initializing a V1 LLM engine (v0.8.4) with config: model='google/paligemma-3b-mix-448', speculative_config=None, tokenizer='google/paligemma-3b-mix-448', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=8192, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='auto', reasoning_backend=None), observabilit

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.


INFO 07-01 05:07:25 [gpu_model_runner.py:1276] Starting to load model google/paligemma-3b-mix-448...
INFO 07-01 05:07:25 [config.py:3466] cudagraph sizes specified by model runner [1, 2, 4, 8, 16, 24, 32, 40, 48, 56, 64, 72, 80, 88, 96, 104, 112, 120, 128, 136, 144, 152, 160, 168, 176, 184, 192, 200, 208, 216, 224, 232, 240, 248, 256, 264, 272, 280, 288, 296, 304, 312, 320, 328, 336, 344, 352, 360, 368, 376, 384, 392, 400, 408, 416, 424, 432, 440, 448, 456, 464, 472, 480, 488, 496, 504, 512] is overridden by config [512, 384, 256, 128, 4, 2, 1, 392, 264, 136, 8, 400, 272, 144, 16, 408, 280, 152, 24, 416, 288, 160, 32, 424, 296, 168, 40, 432, 304, 176, 48, 440, 312, 184, 56, 448, 320, 192, 64, 456, 328, 200, 72, 464, 336, 208, 80, 472, 344, 216, 88, 120, 480, 352, 248, 224, 96, 488, 504, 360, 232, 104, 496, 368, 240, 112, 376]
WARNING 07-01 05:07:25 [gemma.py:60] Gemma's activation function was incorrectly set to exact GeLU in the config JSON file when it was initially released. Changin

Loading safetensors checkpoint shards:   0% Completed | 0/3 [00:00<?, ?it/s]


INFO 07-01 05:07:29 [loader.py:458] Loading weights took 2.64 seconds
INFO 07-01 05:07:29 [gpu_model_runner.py:1291] Model loading took 5.4612 GiB and 3.704288 seconds


...

INFO 07-01 05:07:29 [gpu_model_runner.py:1560] Encoder cache will be initialized with a budget of 8192 tokens, and profiled with 8 image items of the maximum feature size.


You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.


INFO 07-01 05:07:37 [backends.py:416] Using cache directory: /home/rajeshbht19/.cache/vllm/torch_compile_cache/f7e87dda0e/rank_0_0 for vLLM's torch.compile
INFO 07-01 05:07:37 [backends.py:426] Dynamo bytecode transform time: 6.71 s
INFO 07-01 05:07:37 [backends.py:115] Directly load the compiled graph for shape None from the cache
INFO 07-01 05:07:41 [monitor.py:33] torch.compile takes 6.71 s in total
INFO 07-01 05:07:42 [kv_cache_utils.py:634] GPU KV cache size: 544,576 tokens
INFO 07-01 05:07:42 [kv_cache_utils.py:637] Maximum concurrency for 8,192 tokens per request: 66.48x
INFO 07-01 05:08:06 [gpu_model_runner.py:1626] Graph capturing finished in 23 secs, took 1.10 GiB
INFO 07-01 05:08:06 [core.py:163] init engine (profile, create kv cache, warmup model) took 36.39 seconds


You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.


INFO 07-01 05:08:06 [core_client.py:435] Core engine process 0 ready.


In [16]:
prompt = "ocr"

image = Image.open("password.jpg")

sampling_params = SamplingParams(
        temperature=0.0,
        max_tokens=512
)

start_time = time.time()

outputs = llm.generate({
        "prompt": f"{prompt}",
        "multi_modal_data": {
            "image": image
        },
    
    },
    sampling_params=sampling_params
)

# End timing
end_time = time.time()
inference_time = end_time - start_time

for o in outputs:
    generated_text = o.outputs[0].text
    print(generated_text)

print(f"\n⏱️ Inference time: {inference_time:.3f} seconds")

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Ashley Hotel West Coast
WIFI Internet Service
Username: fqqp
Password: aaeu
Traffic: 1 GB
Price: 0.00
Validity: 1w
Starts: Now
Shared Users: 4
27/02/2019 11:03:15
To logout or check
status please type in
http://hello.wifi
Thank you !

⏱️ Inference time: 2.012 seconds
